## Imports

In [1]:
import os
import json
from datetime import datetime
import numpy as np 
import torch 
from torch.utils.data import Dataset, DataLoader

## Global varibles

In [2]:
DATASET = '1K'
BATCH_SIZE = 16
OLD_MODEL = 'logreg-20230516T134843Z.txt'
INPUT_SIZE = 6
OUTPUT_SIZE = 1
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 0.0001
EPOCHS = 50

## Load data
Note that this data is balanced by undersampling the negative class (non-sar txs).

In [3]:
class Dataset(Dataset):
    def __init__(self, X, y):
        self.X = X, 
        self.y = y
    
    def __getitem__(self, idx):
        X = self.X[0][idx].astype('float32') 
        y = self.y[idx].astype('float32')
        return X, y
    
    def __len__(self):
        return len(self.y)

# load data
X_train = np.genfromtxt(f'./datasets/{DATASET}/X_train.csv', delimiter=',', skip_header=1)
y_train = np.genfromtxt(f'./datasets/{DATASET}/y_train.csv', delimiter=',', skip_header=1)
X_test = np.genfromtxt(f'./datasets/{DATASET}/X_test.csv', delimiter=',', skip_header=1)
y_test = np.genfromtxt(f'./datasets/{DATASET}/y_test.csv', delimiter=',', skip_header=1)

# make datasets
train_set = Dataset(X_train, y_train)
test_set = Dataset(X_test, y_test)

# make dataloaders
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

## Load model

In [4]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_size, output_size)
        self.sigmoid = torch.nn.Sigmoid()
    
    def forward(self, x):
        x = self.linear(x)
        x = self.sigmoid(x)
        return x

def load_model_from_txt(model, file_path=None):
    models = os.listdir('models')
    if file_path is None:
        if not models:
            print(f'no models found')
        else:
            file = max(models, key=lambda x: datetime.strptime(x.split('.')[0].split('-')[1], '%Y%m%dT%H%M%SZ'))
            file_path = f'./models/{file}'
            print(f'loaded lateste model: {file_path}')
            with open(file_path, 'r') as f:
                state_dict_json = json.load(f)
            state_dict = {k: torch.tensor(v) for k, v in state_dict_json.items()}
            model.load_state_dict(state_dict)
    else:
        try:
            with open(file_path, 'r') as f:
                state_dict_json = json.load(f)
                print(f'loaded model: {file_path}')
                state_dict = {k: torch.tensor(v) for k, v in state_dict_json.items()}
                model.load_state_dict(state_dict)
        except:
            print(f'model {file_path} not found')
    return model

# init model
model = LogisticRegression(INPUT_SIZE, OUTPUT_SIZE)

# uncoment to load old model
# if file_path is None (defult), the latest model will be loaded
file_path = f'./models/{OLD_MODEL}'
#file_path = None
model = load_model_from_txt(model, file_path)

# move model to device
model.to(DEVICE)

loaded model: ./models/logreg-20230516T134843Z.txt


LogisticRegression(
  (linear): Linear(in_features=6, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

## Train model

In [5]:
def train(model, train_loader):
    optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
    criterion = torch.nn.BCELoss()
    model.train()
    losses = []
    for _ in range(EPOCHS):
        for X_train, y_true in train_loader:
            X_train = X_train.to(DEVICE)
            y_true = y_true.to(DEVICE)
            optimizer.zero_grad()
            y_pred = torch.squeeze(model(X_train))
            loss = criterion(y_pred, y_true)
            loss.backward()
            optimizer.step()        
            losses.append(loss.item())
    loss = sum(losses)/len(losses) 
    return model, loss

def test(model, test_loader):
    criterion = torch.nn.BCELoss()
    model.eval()
    losses = []
    with torch.no_grad():
        for X_test, y_true in test_loader:
            X_test = X_test.to(DEVICE)
            y_true = y_true.to(DEVICE)
            y_pred = torch.squeeze(model(X_test))
            loss = criterion(y_pred, y_true)
            losses.append(loss.item())
    loss = sum(losses)/len(losses) 
    return loss

test_loss_old = test(model, test_loader)
model, train_loss = train(model, train_loader)
test_loss = test(model, test_loader)

print(f'Train loss: {train_loss}')
print(f'Test loss from old model: {test_loss_old}')
print(f'Test loss from new model: {test_loss}')

Train loss: 42.30904855183193
Test loss from old model: 43.281049728393555
Test loss from new model: 37.13622283935547


## Save new model

In [6]:
def save_model_to_txt(model, file_path):
    state_dict = model.state_dict()
    state_dict_json = {k: v.tolist() for k, v in state_dict.items()}
    with open(file_path, 'w') as f:
        json.dump(state_dict_json, f)

new_model_name = 'logreg' + '-' + datetime.utcnow().strftime('%Y%m%dT%H%M%S') + 'Z' + '.txt'
save_model_to_txt(model, f'./models/{new_model_name}')